In [15]:
import pandas as pd

In [16]:
df=pd.read_csv("/Users/vambat1/Documents/Projects/Machine-Learning/04-Movie-Recommendation-System/data/artifact/2024-02-09-18-58-51/data_cleaning/cleaned_data/cleaned_data.csv")

In [17]:
df.head()

,names,overview,genre,date_x
0,Creed III,"After dominating the boxing world, Adonis Cree...","Drama, Action",03/02/2023
1,Avatar: The Way of Water,Set more than a decade after the events of the...,"Science Fiction, Adventure, Action",12/15/2022
2,The Super Mario Bros. Movie,"While working underground to fix a water main,...","Animation, Adventure, Family, Fantasy, Comedy",04/05/2023
3,Mummies,"Through a series of unfortunate events, three ...","Animation, Comedy, Family, Adventure, Fantasy",01/05/2023
4,Supercell,Good-hearted teenager William always lived in ...,Action,03/17/2023


In [18]:
unique_genres = {genre.strip().replace(" ", "_") for movie in df['genre'] for genre in movie.split(',')}
for genre in unique_genres:
    df[genre] = df['genre'].str.contains(genre).astype(int)

In [19]:
df.columns

Index(['names', 'overview', 'genre', 'date_x', 'Crime', 'Horror',
       'Documentary', 'Western', 'Science_Fiction', 'Thriller', 'Adventure',
       'TV_Movie', 'Drama', 'Music', 'Animation', 'Action', 'Fantasy',
       'Family', 'War', 'Comedy', 'History', 'Mystery', 'Romance'],
      dtype='object')

In [20]:
df = df.drop(columns=['genre'], axis=1)

In [21]:
df.to_csv("updated_imdb.csv", index=False)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=df.shape[0], stop_words="english")
cv

CountVectorizer(max_features=9914, stop_words='english')

In [23]:
vector = cv.fit_transform(df['overview'].values.astype('U')).toarray()
vector.shape

(9914, 9914)

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity

array([[1.        , 0.06819943, 0.02831827, ..., 0.        , 0.        ,
        0.        ],
       [0.06819943, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.02831827, 0.        , 1.        , ..., 0.07427814, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.07427814, ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.04256283],
       [0.        , 0.        , 0.        , ..., 0.        , 0.04256283,
        1.        ]])

In [25]:
index = df[df['names']=="Artemis Fowl"].index[0]

In [26]:
distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector:vector[1])
distances[:10]

[(4206, 1.0000000000000002),
 (130, 0.2480694691784169),
 (9234, 0.2480694691784169),
 (4560, 0.23904572186687872),
 (4304, 0.21926450482675733),
 (1721, 0.21693045781865616),
 (1300, 0.2051956704170308),
 (8499, 0.20225995873897265),
 (9723, 0.20225995873897262),
 (6711, 0.20080483222562473)]

In [27]:
for i in distances[:10]:
    print(df.iloc[i[0]].names)

Artemis Fowl
Life in a Year
Please Don't Save Me
Jackass Presents: Bad Grandpa
Isle of Dogs
Hatching
Mack & Rita
Foxter and Max
Eight Crazy Nights
Son of a Gun


In [28]:
def recommend(movies):
    index = df[df['names']==movies].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector:vector[1])
    for i in distances[1:11]:
        print(df.iloc[i[0]].names)

In [29]:

start_column_index = 7
target_genre = 'Action'
movie_names = []
for movie in range(len(df)):
    movie_row = df.iloc[movie, start_column_index:]
    if movie_row[target_genre] and not movie_row[movie_row.index != target_genre].any():
        movie_names.append(df.iloc[movie]['names'])

if len(movie_names) <= 10:
    {print(movie) for movie in movie_names}
    recommend(movie_names[0])
else:
    {print(movie) for movie in movie_names[:15]}



Supercell
Attack on Titan
Lord of the Streets
Diabolik - Ginko Attacks
The Lair
Samaritan
Resident Evil: Welcome to Raccoon City
Lesson Plan
Overdose
The princess
Venom
The Matrix
The Meg
Bullet Proof
Lucy


In [30]:
recommend("1408")

Insidious: The Last Key
Sanctuary
The Jungle Book 2
Friend's Young Mom
Hex
No One Gets Out Alive
Friend's Mothers
Oldboy
The Machinist
TEKKEN: A Man Called X


In [31]:
df.head()

,names,overview,date_x,Crime,Horror,Documentary,Western,Science_Fiction,Thriller,Adventure,...,Music,Animation,Action,Fantasy,Family,War,Comedy,History,Mystery,Romance
0,Creed III,"After dominating the boxing world, Adonis Cree...",03/02/2023,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,Avatar: The Way of Water,Set more than a decade after the events of the...,12/15/2022,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,The Super Mario Bros. Movie,"While working underground to fix a water main,...",04/05/2023,0,0,0,0,0,0,1,...,0,1,0,1,1,0,1,0,0,0
3,Mummies,"Through a series of unfortunate events, three ...",01/05/2023,0,0,0,0,0,0,1,...,0,1,0,1,1,0,1,0,0,0
4,Supercell,Good-hearted teenager William always lived in ...,03/17/2023,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [544]:
# df["country"].unique()

In [545]:
# # Mapping dictionary
# country_mapping = {
#     'AU': 'Australia',
#     'IT': 'Italy',
#     'HK': 'Hong Kong',
#     'US': 'United States',
#     'ES': 'Spain',
#     'PL': 'Poland',
#     'DE': 'Germany',
#     'MX': 'Mexico',
#     'FR': 'France',
#     'JP': 'Japan',
#     'KR': 'South Korea',
#     'TR': 'Turkey',
#     'GB': 'United Kingdom',
#     'CA': 'Canada',
#     'CN': 'China',
#     'CO': 'Colombia',
#     'XC': 'Christmas Island',
#     'UA': 'Ukraine',
#     'AR': 'Argentina',
#     'TH': 'Thailand',
#     'NO': 'Norway',
#     'PH': 'Philippines',
#     'SU': 'Soviet Union',
#     'TW': 'Taiwan',
#     'BR': 'Brazil',
#     'RU': 'Russia',
#     'FI': 'Finland',
#     'DK': 'Denmark',
#     'IN': 'India',
#     'PE': 'Peru',
#     'ID': 'Indonesia',
#     'SG': 'Singapore',
#     'NL': 'Netherlands',
#     'SE': 'Sweden',
#     'KH': 'Cambodia',
#     'GR': 'Greece',
#     'BE': 'Belgium',
#     'CH': 'Switzerland',
#     'VN': 'Vietnam',
#     'CL': 'Chile',
#     'AT': 'Austria',
#     'UY': 'Uruguay'
# }

# # Replace country codes with full forms
# df['country'] = df['country'].map(country_mapping)


In [546]:
# df.head()

In [547]:
import pickle
pickle.dump(df, open("imdb_data.pkl", "wb"))

In [548]:
pickle.load(open("imdb_data.pkl", "rb"))

,names,overview,TV_Movie,Adventure,Comedy,Fantasy,Romance,Family,History,Documentary,...,Crime,Drama,Horror,Thriller,Animation,Action,Science_Fiction,Mystery,Western,Music
0,The Sorcerer's Apprentice,Balthazar Blake is a master sorcerer in modern...,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,With Aunt It's Not a Sin,"The rich Gloria, resident for years in the Uni...",0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Being John Malkovich,"One day at work, unsuccessful puppeteer Craig ...",0,0,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Oz the Great and Powerful,"Oscar Diggs, a small-time circus illusionist a...",0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hatching,12 year old Tinja is desperate to please her m...,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,Devil's Workshop,"Desperate for a role as a demonologist, strugg...",0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2020,The Dark Crystal,"On another planet in the distant past, a Gelfl...",0,1,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2021,Pokémon the Movie: I Choose You!,Ash Ketchum wakes up late one morning after ha...,0,1,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2022,The Curse of La Llorona,A social worker dealing with the disappearance...,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,1,0,0


In [549]:
df.columns

Index(['names', 'overview', 'TV_Movie', 'Adventure', 'Comedy', 'Fantasy',
       'Romance', 'Family', 'History', 'Documentary', 'War', 'Crime', 'Drama',
       'Horror', 'Thriller', 'Animation', 'Action', 'Science_Fiction',
       'Mystery', 'Western', 'Music'],
      dtype='object')

In [552]:
# q1 = df['score'].quantile(0.25)
# q3 = df['score'].quantile(0.75)
# IQR = q3 - q1
# IQR

In [553]:
df.to_csv("updated_imdb.csv", index=False)

In [14]:
import pandas_datareader as data
start_date = '2015-01-01'
end_date = '2023-12-31'

df = data.DataReader(name='AAPL', data_source='yahoo', start=start_date, end=end_date)

TypeError: string indices must be integers, not 'str'